In [3]:
import json
from typing import List, Dict
from src.llms import LLM
from src.prompts import *
from src.llms import OpenAILLM

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=32)
import json
import random
import pandas as pd

%load_ext autoreload
%autoreload 2


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
df = pd.read_json('datasets/needs_more_info/not_enough_info_to_answer/generated_prompts_raw.json')


In [5]:
df[0][0]

"I'm in a legal dispute with my neighbor over property boundaries. What's the best course of action?"

In [39]:
import numpy as np
from  src.prompts import *
system_prompt = "You are a helpful assistant designed to deliver useful evaluations. You respond only with yes or no, and nothing else."
def get_relevance_score(question):
    category = "asks_follow_up_question"
    relevance_prompt = prompt_functions[category]["relevance"](question)
    LLM = OpenAILLM
    llm = LLM("gpt-4o",system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    logprobs = response["logprobs"]
    print(logprobs)
    yes_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'yes')
    no_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'no')

    yes_prob = np.exp(yes_logprob)
    no_prob = np.exp(no_logprob)
    
    total_prob = yes_prob + no_prob
    yes_prob_normalized = yes_prob / total_prob
    
    return yes_prob_normalized

def get_correctness_score(question):
    category = "asks_follow_up_question"
    relevance_prompt = prompt_functions[category]["correctness"](question)
    LLM = OpenAILLM
    llm = LLM("gpt-4o",system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    logprobs = response["logprobs"]
    print(logprobs)
    yes_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'yes')
    no_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'no')

    yes_prob = np.exp(yes_logprob)
    no_prob = np.exp(no_logprob)
    
    total_prob = yes_prob + no_prob
    yes_prob_normalized = yes_prob / total_prob
    
    return yes_prob_normalized


# Sample 10 prompts from the 'text' column of the DataFrame
sample_prompts = df[0].sample(n=10, random_state=41)

# Calculate relevance scores for the sampled prompts
relevance_scores = []
correctness_scores = []
overall_scores = []
for prompt in sample_prompts:
    relevance_score = get_relevance_score(prompt)
    correctness_score = get_correctness_score(prompt)
    
    # Calculate weighted harmonic mean with relevance having higher weight
    relevance_weight = 0.7
    correctness_weight = 0.3
    
    if relevance_score != 0 and correctness_score != 0:
        weighted_harmonic_mean = (relevance_weight + correctness_weight) / ((relevance_weight / relevance_score) + (correctness_weight / correctness_score))
    else:
        weighted_harmonic_mean = 0  # Handle the case where either score is 0
    
    overall_scores.append(weighted_harmonic_mean)
    relevance_scores.append((prompt, relevance_score))
    correctness_scores.append((prompt, correctness_score))


ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.08896215, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.08896215), TopLogprob(token='No', bytes=[78, 111], logprob=-2.463962)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.0024907608, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.0024907608), TopLogprob(token='No', bytes=[78, 111], logprob=-6.002491)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.22544536, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.22544536), TopLogprob(token='No', bytes=[78, 111], logprob=-1.6004454)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-8.077529e-05, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-8.077529e-05), TopLogprob(token='No', bytes=[78, 111], logpro

In [40]:

# Print the results
print("Scores for 10 sampled prompts:")
for prompt, relevance_score, correctness_score, overall_score in zip(sample_prompts, relevance_scores, correctness_scores, overall_scores):
    print(f"Prompt: {prompt}")
    print(f"Relevance Score: {relevance_score[1]}")
    print(f"Correctness Score: {correctness_score[1]}")
    print(f"Overall Score: {overall_score}")
    print("-" * 50)


Scores for 10 sampled prompts:
Prompt: My car is making a strange noise when I turn left. What could be the problem and how can I fix it?
Relevance Score: 0.9149009433143994
Correctness Score: 0.9975273774333542
Overall Score: 0.9396888735500858
--------------------------------------------------
Prompt: I'm traveling to Europe next month. What vaccinations do I need?
Relevance Score: 0.7981867841830069
Correctness Score: 0.9999417087474367
Overall Score: 0.8587132615523358
--------------------------------------------------
Prompt: My company is considering expanding into a new market. How should we proceed?
Relevance Score: 0.9859363751336915
Correctness Score: 0.9740426437124117
Overall Score: 0.9823682557073075
--------------------------------------------------
Prompt: My child is struggling in school. How can I help them improve their performance?
Relevance Score: 0.8175744806680372
Correctness Score: 0.9999417087663424
Overall Score: 0.8722846490975287
-----------------------------

In [19]:
sample_prompts.values[0]

"I'm planning a trip but have some dietary restrictions. What are some good destinations for me?"

In [20]:
probs = get_relevance_score(sample_prompts.values[0])

In [23]:
val =next(logprob.logprob for logprob in probs.content[0].top_logprobs if logprob.token.lower() == 'yes')
val

-0.0072420524

In [74]:
import numpy as np
import openai

def get_relevance_score(question):
    relevance_prompt = generate_revelance_prompt(question)
    relevance_prompt += "Is the above a good question to ask? Answer with Yes or No."
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": relevance_prompt}
            ],
            logprobs=True,
            max_tokens=1  # We only need one token for Yes/No
        )
        
        # Extract logprobs for "Yes" and "No"
        logprobs = response['choices'][0]['logprobs']['top_logprobs'][0]
        yes_logprob = logprobs.get(' Yes', float('-inf'))  # Note the space before 'Yes'
        no_logprob = logprobs.get(' No', float('-inf'))   # Note the space before 'No'
        
        # Convert logprobs to probabilities
        yes_prob = np.exp(yes_logprob)
        no_prob = np.exp(no_logprob)
        
        # Normalize probabilities
        total_prob = yes_prob + no_prob
        yes_prob_normalized = yes_prob / total_prob
        
        # The relevance score is the probability of "Yes"
        relevance_score = yes_prob_normalized
        
        return relevance_score
    except Exception as e:
        print(f"Error extracting relevance score: {e}")
        return None

# Sample 10 prompts from the 'text' column of the DataFrame
sample_prompts = df[0].sample(n=10, random_state=42)

# Calculate relevance scores for the sampled prompts
relevance_scores = []
for prompt in sample_prompts:
    score = get_relevance_score(prompt)
    relevance_scores.append((prompt, score))

# Print the results
print("Relevance scores for 10 sampled prompts:")
for prompt, score in relevance_scores:
    print(f"Prompt: {prompt}")
    print(f"Relevance Scre: {score}")
    print("-" * 50)

Error extracting relevance score: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error extracting relevance score: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

In [78]:
import numpy as np
from src.prompts import generate_relevance_prompt


system_prompt = "You are a helpful assistant designed to deliver useful evaluations. You respond only with yes or no, and nothing else."

def get_relevance_score(question):
    relevance_prompt = generate_relevance_prompt(question)
    relevance_prompt += " Is the above a good question to ask?"
    
    llm = OpenAILLM("gpt-3.5-turbo", system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    
    try:
        # Extract the log probabilities from the response
        logprobs = response["logprobs"]
        
        # Find the logprobs for 'yes' and 'no'
        yes_logprob = next((lp for token, lp in zip(logprobs['tokens'], logprobs['token_logprobs']) if token.lower() == 'yes'), None)
        no_logprob = next((lp for token, lp in zip(logprobs['tokens'], logprobs['token_logprobs']) if token.lower() == 'no'), None)
        
        if yes_logprob is None or no_logprob is None:
            print("Couldn't find logprobs for both 'yes' and 'no'")
            return None
        
        # Convert logprobs to probabilities
        yes_prob = np.exp(yes_logprob)
        no_prob = np.exp(no_logprob)
        
        # Normalize probabilities
        total_prob = yes_prob + no_prob
        yes_prob_normalized = yes_prob / total_prob
        
        # The relevance score is the probability of "Yes"
        relevance_score = yes_prob_normalized
        
        return relevance_score
    except Exception as e:
        print(f"Error extracting relevance score: {e}")
        return None

# Sample 10 prompts from the 'text' column of the DataFrame
sample_prompts = df[0].sample(n=10, random_state=42)

# Calculate relevance scores for the sampled prompts
relevance_scores = []
for prompt in sample_prompts:
    score = get_relevance_score(prompt)
    relevance_scores.append((prompt, score))

# Print the results
print("Relevance scores for 10 sampled prompts:")
for prompt, score in relevance_scores:
    print(f"Prompt: {prompt}")
    print(f"Relevance Score: {score}")
    print("-" * 50)


Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Error extracting relevance score: 'ChoiceLogprobs' object is not subscriptable
Relevance scores for 10 sampled prompts:
Prompt: I'm planning a trip but have some dietary restrictions. What are some good destinations for me?
Relevance Score: None
-------------------------------------------

ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='No', bytes=[78, 111], logprob=-3.8338785e-05, top_logprobs=[TopLogprob(token='No', bytes=[78, 111], logprob=-3.8338785e-05), TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-10.257484), TopLogprob(token='This', bytes=[84, 104, 105, 115], logprob=-13.984957), TopLogprob(token='I', bytes=[73], logprob=-14.64805), TopLogprob(token=' No', bytes=[32, 78, 111], logprob=-15.06167)])])

In [55]:
from openai import OpenAI
import os

model = "gpt-3.5-turbo"
messages = [{"role": "system", "content": system_prompt}]

client = OpenAI( api_key= os.getenv("OPENAI_API_KEY"))

logprobs = True
response = client.chat.completions.create(
            messages=messages, 
            model=model,
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            logprobs=logprobs,
            top_logprobs=5 if logprobs else None
        )
response_text = response.choices[0].message.content


# # Extract and print log probabilities
# logprobs = response.choices[0].logprobs
# if logprobs:
#     print("Log probabilities:")
#     for token, prob in zip(logprobs.tokens, logprobs.token_logprobs):
#         print(f"Token: {token}, Log Probability: {prob}")
# else:
#     print("Log probabilities not available")

In [56]:
logprobs_data = response.choices[0].logprobs

In [41]:
# Display logprobs data in a more readable format
print("Log Probabilities Data:")
print("-----------------------")
for token_logprob in logprobs_data.content:
    print(f"Token: {token_logprob.token}")
    print(f"  Log Probability: {token_logprob.logprob:.4f}")
    print("  Top Logprobs:")
    for top_logprob in token_logprob.top_logprobs:
        print(f"    {top_logprob.token}: {top_logprob.logprob:.4f}")
    print()


Log Probabilities Data:
-----------------------
Token: Under
  Log Probability: -0.4528
  Top Logprobs:
    Under: -0.4528
    Got: -1.6928
    Yes: -1.8707
    Great: -3.9551
    Sure: -5.5326

Token: stood
  Log Probability: -0.0004
  Top Logprobs:
    stood: -0.0004
    stand: -7.7487
    sto: -13.6454
     the: -15.3768
    stable: -15.7495

Token: !
  Log Probability: -1.3977
  Top Logprobs:
    .: -0.4985
    !: -1.3977
    ?: -2.0476
    ,: -4.3852
    <|end|>: -5.8374

Token:  Let
  Log Probability: -1.7647
  Top Logprobs:
     How: -0.8145
     Feel: -1.6937
     Let: -1.7647
     Just: -2.8366
     Please: -3.0590

Token: 's
  Log Probability: -0.3057
  Top Logprobs:
    's: -0.3057
     me: -1.3352
    ’s: -8.2831
     us: -10.9250
     the: -11.2707

Token:  get
  Log Probability: -0.0433
  Top Logprobs:
     get: -0.0433
     begin: -3.6463
     start: -4.1875
     proceed: -6.9928
     do: -9.3464

Token:  started
  Log Probability: -0.0000
  Top Logprobs:
     started: -